### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `SDK_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [1]:
pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -e ..

Obtaining file:///Users/thibautpatel/github/chainlit-sdks/python-client
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chainlit-sdk
    Found existing installation: chainlit-sdk 0.1.0
    Uninstalling chainlit-sdk-0.1.0:
      Successfully uninstalled chainlit-sdk-0.1.0
  Running setup.py develop for chainlit-sdk

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Example

In [4]:
import asyncio
import json
import uuid

from chainlit_sdk import Chainlit
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

sdk = Chainlit(batch_size=2)
sdk.instrument_openai()

In [5]:
# Start a conversation

thread_id = uuid.uuid4()

welcome_message = "What's your name? "
with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.set_parameter("content", welcome_message)
    step.set_parameter("role", "assistant")

text = input(welcome_message)

What's your name?  Joe


In [6]:
# reply with an openai completion

with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.set_parameter("content", text)
    step.set_parameter("role", "user")
    with sdk.observer.step(type="run", thread_id=thread_id) as step:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
                },
                {
                    "role": "assistant",
                    "content": welcome_message,
                },
                {
                    "role": "user",
                    "content": text,
                },
            ],
        )
        with sdk.observer.step(type="message", thread_id=thread_id) as step:
            print("")
            print(completion.choices[0].message.content)
            step.set_parameter("content", completion.choices[0].message.content)
            step.set_parameter("role", "assistant")



Keep pushing forward, Joe, because you are capable of achieving incredible things.


In [7]:
sdk.wait_until_queue_empty() # wait for all msgs to be in the cloud instance

In [8]:
# Get the thread

print("\nSearching for the thread", thread_id, "...")
steps = await sdk.api.get_thread(id=str(thread_id))

thread = steps

print(json.dumps(thread, indent=2))


Searching for the thread c6d9d561-0b37-4cfa-9b09-95bcbd19a394 ...
{
  "data": {
    "thread": {
      "id": "c6d9d561-0b37-4cfa-9b09-95bcbd19a394",
      "steps": [
        {
          "id": "8b598e09-479d-4a1f-acce-0a434778e84a",
          "parentId": null,
          "startTime": 1696898389,
          "endTime": 1696898389,
          "type": "MESSAGE",
          "input": null,
          "output": null,
          "metadata": {},
          "tags": [],
          "operatorName": null,
          "operatorRole": null,
          "generation": null,
          "feedback": null
        },
        {
          "id": "220a1625-1dd7-4803-adbc-6be6ce217a1c",
          "parentId": null,
          "startTime": 1696898396,
          "endTime": 1696898397,
          "type": "LLM",
          "input": null,
          "output": null,
          "metadata": {},
          "tags": [],
          "operatorName": null,
          "operatorRole": null,
          "feedback": null,
          "generation": null
     